In [1]:
import sys
sys.path.append('..')
from LL_functions import *
import utils #For getting the filename
import os
import matplotlib.pyplot as plt
import matplotlib

#Set plot fonts
font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 16}
matplotlib.rc('font', **font)
print_bold = '\033[1m'

## Hyperparameters
These hyperparameters were determined by a hyperparameter search. The hyperparameters with two values correspond to \[baseline value, ps value\].

In [2]:
#Dataset
n_workers    = 24
n_bs         = [2000, 1000]
n_bs_incep   = [2000, 1000]
G_acc        = [1, 2]
D_acc        = [1, 2]
shuffle      = [True, False]

#Optimization
n_D_step     = [5, 5]
G_lr         = [0.005477874786325012, 0.005477874786325012]
D_lr         = 0.0011634802123515887
G_lr_decay   = [1.00064523807154, 1.00064523807154]
D_lr_decay   = 1.0001567511690173
n_epochs     = 1000

#Misc
n_test_every = 50
n_save_every = 50
train_fn     = ['GAN', 'PS']

#Architecture
G_ch         = 8
D_ch         = [28, 28]
dim_z_vals   = np.flip(2**(np.arange(10) + 2)) #Do largest first

#Seed
seed_vals    = [0]

#Resume
resume       = False

## Useful functions

In [3]:
#Function for formating arguments
def format_args(_n_bs, _n_bs_incep, _n_workers, _G_acc, _D_acc, _n_D_step, _G_lr, _D_lr, _n_epochs, shuffle, train_fn,
                _G_lr_decay, _D_lr_decay, _G_ch, _D_ch, _n_test_every, _n_save_every, _seed, _resume, _dim_z):

    #Args
    args_dataset  = ['--dataset', 'celeba_64_cc_SUB10000', '--parallel', '--num_workers', str(_n_workers), 
                     '--batch_size', str(_n_bs), '--inception_batchsize', str(_n_bs_incep)]
    if shuffle:
        args_dataset += ['--shuffle']
    args_acc      = ['--num_G_accumulations', str(_G_acc), '--num_D_accumulations', str(_D_acc)]
    args_optim    = ['--num_D_steps', str(_n_D_step), '--G_lr', str(_G_lr), '--D_lr', str(_D_lr), 
                     '--D_B2', '0.900', '--G_B2', '0.900', '--num_epochs', str(_n_epochs),
                     '--G_lr_decay', str(_G_lr_decay), '--D_lr_decay', str(_D_lr_decay)]
    args_arch     = ['--G_attn', '0', '--D_attn', '0', '--G_nl', 'relu', '--D_nl', 'relu', '--G_ortho', '0.0',
                     '--D_thin', '--G_init', 'xavier', '--D_init', 'xavier', '--G_eval_mode', 
                     '--G_ch', str(_G_ch), '--D_ch', str(_D_ch), '--dim_z', str(_dim_z)]
    args_tol      = ['--SN_eps', '1e-8', '--BN_eps', '1e-5', '--adam_eps', '1e-8']
    args_saving   = ['--test_every', str(_n_test_every), '--save_every', str(_n_save_every), 
                     '--num_best_copies', '5', '--num_save_copies', '2', '--seed', str(_seed)]
    args_misc     = ['--name_suffix', 'SNGAN', '--data_root', '/data', '--load_in_mem', '--which_train_fn', train_fn]

    #Combine
    args = args_dataset + args_acc + args_optim + args_arch + args_tol + args_saving + args_misc
    if _resume:
        args += ['--resume']
        
    return args

#This reads the information on the log
def get_log_info(f_log):
    #Read in file
    if os.path.exists(f_log + '_log.jsonl'):
        fid = open(f_log + '_log.jsonl', 'r')
        raw = fid.read()
        fid.close()
    else:
        return

    #Clean up a bit
    raw = raw.split('\n')[:-1]
    
    #Initialize arrays
    n_tests    = len(raw)
    itr        = np.zeros(n_tests, dtype=int)
    IS_mean    = np.zeros(n_tests)
    IS_std     = np.zeros(n_tests)
    FID        = np.zeros(n_tests)
    time_stamp = np.zeros(n_tests)

    #Populate arrays
    for i in range(n_tests):
        itr[i]        = raw[i].split('\"itr\": ')[1].split(',')[0]
        IS_mean[i]    = raw[i].split('\"IS_mean\": ')[1].split(',')[0]
        IS_std[i]     = raw[i].split('\"IS_std\": ')[1].split(',')[0]
        FID[i]        = raw[i].split('\"FID\": ')[1].split(',')[0]
        time_stamp[i] = raw[i].split('\"_stamp\": ')[1].split('}')[0]

    #Transfer arrays over to a dictionary
    result = dict()
    result['itr']        = itr
    result['IS_mean']    = IS_mean
    result['IS_std']     = IS_std
    result['FID']        = FID
    result['time_stamp'] = time_stamp

    return result

def get_log():
    parser      = utils.prepare_parser()
    config      = vars(parser.parse_args())
    f_model     = utils.name_from_config(config)
    log         = get_log_info(os.path.join('logs', f_model))
    return log, f_model

## Run through seed_vals and dim_z_vals for the baseline and PS models

In [ ]:
#Initialize f_model array
f_models = []
  
#Do the baseline and then the ps
for ps in [0,1]:
    #Loop through each seed
    for i in range(len(seed_vals)):
        #Set seed
        seed = seed_vals[i]
        set_seed(seed)

        #Loop through each latent size
        for ii in range(len(dim_z_vals)):
            #Set dim_z
            dim_z = dim_z_vals[ii]

            #Logging
            s_viz = '------------------------------------------------------------\n'
            print(s_viz + s_viz +'SEED = ' + str(seed) + '. DIM_Z = ' + str(dim_z) + '\n' + s_viz + s_viz)

            #Populate arguments
            args = format_args(n_bs[ps], n_bs_incep[ps], n_workers, G_acc[ps], D_acc[ps], n_D_step[ps], G_lr[ps], D_lr, n_epochs, shuffle[ps], train_fn[ps],
                               G_lr_decay[ps], D_lr_decay, G_ch, D_ch[ps], n_test_every, n_save_every, seed, resume, dim_z)
            sys.argv = ['train.py'] + args

            #Get filename
            log, f_model   = get_log()
            f_models.append(f_model)

            #Attempt to resume if possible
            if log is None:
                print('No log. Starting from scratch')
                args = format_args(n_bs[ps], n_bs_incep[ps], n_workers, G_acc[ps], D_acc[ps], n_D_step[ps], G_lr[ps], D_lr, n_epochs, shuffle[ps], train_fn[ps],
                               G_lr_decay[ps], D_lr_decay, G_ch, D_ch[ps], n_test_every, n_save_every, seed, False, dim_z)
                sys.argv = ['train.py'] + args
                exec(open("train.py").read())
            elif log['itr'][-1] < n_epochs:
                print('Log exists, but we are at iteration #', log['itr'][-1], ', continuing training to n_epochs.')
                args = format_args(n_bs[ps], n_bs_incep[ps], n_workers, G_acc[ps], D_acc[ps], n_D_step[ps], G_lr[ps], D_lr, n_epochs, shuffle[ps], train_fn[ps],
                               G_lr_decay[ps], D_lr_decay, G_ch, D_ch[ps], n_test_every, n_save_every, seed, True, dim_z)
                sys.argv = ['train.py'] + args
                exec(open("train.py").read())
            else:
                log, f_model   = get_log()
                print('Done training. FID =',np.nanmin(log['FID']))
                pass

------------------------------------------------------------
------------------------------------------------------------
SEED = 0. DIM_Z = 2048
------------------------------------------------------------
------------------------------------------------------------

Done training. FID = 94.71969604492188
------------------------------------------------------------
------------------------------------------------------------
SEED = 0. DIM_Z = 1024
------------------------------------------------------------
------------------------------------------------------------

Done training. FID = 91.32876586914062
------------------------------------------------------------
------------------------------------------------------------
SEED = 0. DIM_Z = 512
------------------------------------------------------------
------------------------------------------------------------

Done training. FID = 61.3975830078125
------------------------------------------------------------
--------------------

100%|██████████| 10000/10000 [00:05<00:00, 1808.93it/s]


Parallelizing Inception module...
Beginning training at epoch 99...
1/1 (100.00%) (took 0:33) itr: 101, G_loss : +0.93, D_loss_real : +0.22, D_loss_fake : +1.17, G_lr : 5.1e-03, D_lr : 1.1e-03 
1/1 (100.00%) (took 0:09) itr: 102, G_loss : +0.71, D_loss_real : +1.51, D_loss_fake : +0.16, G_lr : 5.1e-03, D_lr : 1.1e-03 
1/1 (100.00%) (took 0:09) itr: 103, G_loss : +0.76, D_loss_real : +2.68, D_loss_fake : +0.00, G_lr : 5.1e-03, D_lr : 1.1e-03 
1/1 (100.00%) (took 0:09) itr: 104, G_loss : +0.52, D_loss_real : +1.47, D_loss_fake : +0.05, G_lr : 5.1e-03, D_lr : 1.1e-03 
1/1 (100.00%) (took 0:09) itr: 105, G_loss : -0.18, D_loss_real : +0.00, D_loss_fake : +3.94, G_lr : 5.1e-03, D_lr : 1.1e-03 
1/1 (100.00%) (took 0:09) itr: 106, G_loss : +2.97, D_loss_real : +0.02, D_loss_fake : +1.35, G_lr : 5.1e-03, D_lr : 1.1e-03 
1/1 (100.00%) (took 0:09) itr: 107, G_loss : +0.18, D_loss_real : +2.55, D_loss_fake : +0.00, G_lr : 5.1e-03, D_lr : 1.1e-03 
1/1 (100.00%) (took 0:10) itr: 108, G_loss : +0.86

/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Calculating Inception Score...
Calculating means and covariances...
Covariances calculated, getting FID...
Itr 150: PYTORCH UNOFFICIAL Inception Score is 1.879 +/- 0.009, PYTORCH UNOFFICIAL FID is 275.6021
IS improved over previous best, saving checkpoint...
Saving weights to weights/BigPS_celeba_64_cc_SUB10000_seed0_Gch8_Dch28_bs1000_dimz2048_nDs5_nDa2_nGa2_Glr5.5e-03_Dlr1.2e-03_GlrDecay1.000645_DlrDecay1.000157_GBB0.900_DBB0.900_Gnlrelu_Dnlrelu_Ginitxavier_Dinitxavier_SNGAN/best1...
1/1 (100.00%) (took 0:37)
1/1 (100.00%) (took 0:09) itr: 151, G_loss : +0.64, D_loss_real : +1.56, D_loss_fake : +0.33, G_lr : 5.0e-03, D_lr : 1.1e-03 
1/1 (100.00%) (took 0:09) itr: 152, G_loss : +1.15, D_loss_real : +0.08, D_loss_fake : +0.23, G_lr : 5.0e-03, D_lr : 1.1e-03 
1/1 (100.00%) (took 0:08) itr: 153, G_loss : +0.45, D_loss_real : +2.09, D_loss_fake : +0.00, G_lr : 5.0e-03, D_lr : 1.1e-03 
1/1 (100.00%) (took 0:09) itr: 154, G_loss : +0.87, D_loss_real : +0.73, D_loss_fake : +0.54, G_lr : 5.0e-

...

1/1 (100.00%) (took 0:11) itr: 507, G_loss : +0.48, D_loss_real : +0.98, D_loss_fake : +0.31, G_lr : 3.9e-03, D_lr : 1.1e-03 
1/1 (100.00%) (took 0:11) itr: 508, G_loss : +0.29, D_loss_real : +1.00, D_loss_fake : +0.13, G_lr : 3.9e-03, D_lr : 1.1e-03 
1/1 (100.00%) (took 0:11) itr: 509, G_loss : +0.21, D_loss_real : +1.38, D_loss_fake : +0.08, G_lr : 3.9e-03, D_lr : 1.1e-03 
1/1 (100.00%) (took 0:10) itr: 510, G_loss : +0.18, D_loss_real : +1.42, D_loss_fake : +0.06, G_lr : 3.9e-03, D_lr : 1.1e-03 
1/1 (100.00%) (took 0:10) itr: 511, G_loss : +0.31, D_loss_real : +1.52, D_loss_fake : +0.05, G_lr : 3.9e-03, D_lr : 1.1e-03 
1/1 (100.00%) (took 0:10) itr: 512, G_loss : +0.89, D_loss_real : +0.41, D_loss_fake : +0.72, G_lr : 3.9e-03, D_lr : 1.1e-03 
1/1 (100.00%) (took 0:11) itr: 513, G_loss : +0.51, D_loss_real : +1.01, D_loss_fake : +0.39, G_lr : 3.9e-03, D_lr : 1.1e-03 
1/1 (100.00%) (took 0:11) itr: 514, G_loss : +0.85, D_loss_real : +0.66, D_loss_fake : +0.57, G_lr : 3.9e-03, D_lr : 1

1/1 (100.00%) (took 0:11) itr: 563, G_loss : -0.11, D_loss_real : +1.61, D_loss_fake : +0.04, G_lr : 3.8e-03, D_lr : 1.1e-03 
1/1 (100.00%) (took 0:11) itr: 564, G_loss : +0.31, D_loss_real : +1.13, D_loss_fake : +0.25, G_lr : 3.8e-03, D_lr : 1.1e-03 
1/1 (100.00%) (took 0:11) itr: 565, G_loss : +0.36, D_loss_real : +1.01, D_loss_fake : +0.21, G_lr : 3.8e-03, D_lr : 1.1e-03 
1/1 (100.00%) (took 0:10) itr: 566, G_loss : +0.18, D_loss_real : +1.18, D_loss_fake : +0.07, G_lr : 3.8e-03, D_lr : 1.1e-03 
1/1 (100.00%) (took 0:11) itr: 567, G_loss : +0.20, D_loss_real : +1.65, D_loss_fake : +0.03, G_lr : 3.8e-03, D_lr : 1.1e-03 
1/1 (100.00%) (took 0:10) itr: 568, G_loss : +0.10, D_loss_real : +1.46, D_loss_fake : +0.07, G_lr : 3.8e-03, D_lr : 1.1e-03 
1/1 (100.00%) (took 0:10) itr: 569, G_loss : +0.22, D_loss_real : +1.35, D_loss_fake : +0.13, G_lr : 3.8e-03, D_lr : 1.1e-03 
1/1 (100.00%) (took 0:11) itr: 570, G_loss : -0.19, D_loss_real : +1.05, D_loss_fake : +0.16, G_lr : 3.8e-03, D_lr : 1

1/1 (100.00%) (took 0:10) itr: 621, G_loss : +0.66, D_loss_real : +0.02, D_loss_fake : +1.76, G_lr : 3.7e-03, D_lr : 1.1e-03 
1/1 (100.00%) (took 0:11) itr: 622, G_loss : +0.09, D_loss_real : +1.42, D_loss_fake : +0.12, G_lr : 3.7e-03, D_lr : 1.1e-03 
1/1 (100.00%) (took 0:11) itr: 623, G_loss : +0.42, D_loss_real : +0.98, D_loss_fake : +0.31, G_lr : 3.7e-03, D_lr : 1.1e-03 
1/1 (100.00%) (took 0:10) itr: 624, G_loss : -0.14, D_loss_real : +1.24, D_loss_fake : +0.08, G_lr : 3.7e-03, D_lr : 1.1e-03 
1/1 (100.00%) (took 0:10) itr: 625, G_loss : +0.85, D_loss_real : +0.10, D_loss_fake : +1.54, G_lr : 3.7e-03, D_lr : 1.1e-03 
1/1 (100.00%) (took 0:11) itr: 626, G_loss : +0.14, D_loss_real : +1.80, D_loss_fake : +0.01, G_lr : 3.7e-03, D_lr : 1.1e-03 
1/1 (100.00%) (took 0:10) itr: 627, G_loss : -0.06, D_loss_real : +1.33, D_loss_fake : +0.08, G_lr : 3.7e-03, D_lr : 1.1e-03 
1/1 (100.00%) (took 0:10) itr: 628, G_loss : -0.16, D_loss_real : +1.22, D_loss_fake : +0.15, G_lr : 3.7e-03, D_lr : 1

1/1 (100.00%) (took 0:10) itr: 679, G_loss : -0.06, D_loss_real : +0.95, D_loss_fake : +0.18, G_lr : 3.5e-03, D_lr : 1.0e-03 
1/1 (100.00%) (took 0:10) itr: 680, G_loss : +0.91, D_loss_real : +0.23, D_loss_fake : +1.14, G_lr : 3.5e-03, D_lr : 1.0e-03 
1/1 (100.00%) (took 0:11) itr: 681, G_loss : +0.16, D_loss_real : +1.15, D_loss_fake : +0.25, G_lr : 3.5e-03, D_lr : 1.0e-03 
1/1 (100.00%) (took 0:11) itr: 682, G_loss : +0.37, D_loss_real : +1.83, D_loss_fake : +0.03, G_lr : 3.5e-03, D_lr : 1.0e-03 
1/1 (100.00%) (took 0:10) itr: 683, G_loss : +0.21, D_loss_real : +1.40, D_loss_fake : +0.09, G_lr : 3.5e-03, D_lr : 1.0e-03 
1/1 (100.00%) (took 0:10) itr: 684, G_loss : +0.04, D_loss_real : +0.89, D_loss_fake : +0.25, G_lr : 3.5e-03, D_lr : 1.0e-03 
1/1 (100.00%) (took 0:11) itr: 685, G_loss : +0.31, D_loss_real : +1.37, D_loss_fake : +0.16, G_lr : 3.5e-03, D_lr : 1.0e-03 
1/1 (100.00%) (took 0:10) itr: 686, G_loss : +0.21, D_loss_real : +1.11, D_loss_fake : +0.20, G_lr : 3.5e-03, D_lr : 1

1/1 (100.00%) (took 0:11) itr: 737, G_loss : +0.18, D_loss_real : +1.24, D_loss_fake : +0.06, G_lr : 3.4e-03, D_lr : 1.0e-03 
1/1 (100.00%) (took 0:10) itr: 738, G_loss : -0.07, D_loss_real : +1.27, D_loss_fake : +0.09, G_lr : 3.4e-03, D_lr : 1.0e-03 
1/1 (100.00%) (took 0:11) itr: 739, G_loss : -0.02, D_loss_real : +1.19, D_loss_fake : +0.11, G_lr : 3.4e-03, D_lr : 1.0e-03 
1/1 (100.00%) (took 0:10) itr: 740, G_loss : +0.66, D_loss_real : +0.06, D_loss_fake : +1.65, G_lr : 3.4e-03, D_lr : 1.0e-03 
1/1 (100.00%) (took 0:11) itr: 741, G_loss : +0.34, D_loss_real : +0.53, D_loss_fake : +0.31, G_lr : 3.4e-03, D_lr : 1.0e-03 
1/1 (100.00%) (took 0:10) itr: 742, G_loss : +0.19, D_loss_real : +0.98, D_loss_fake : +0.11, G_lr : 3.4e-03, D_lr : 1.0e-03 
1/1 (100.00%) (took 0:10) itr: 743, G_loss : +0.87, D_loss_real : +0.25, D_loss_fake : +1.07, G_lr : 3.4e-03, D_lr : 1.0e-03 
1/1 (100.00%) (took 0:10) itr: 744, G_loss : -0.01, D_loss_real : +1.64, D_loss_fake : +0.04, G_lr : 3.4e-03, D_lr : 1

In [ ]:
FID     = []
IS_mean = []
IS_std  = []
for i in range(len(f_models)):
    #Get the log
    f_model = f_models[i]
    f_log   = os.path.join('logs',f_models[i]) 
    log     = get_log_info(f_log)
    
    #Get the performance metrics
    FID.append(log['FID'])
    IS_mean.append(log['IS_mean'])
    IS_std.append(log['IS_std'])
    
    #Get the latent dimension
    dim_z = dim_z_vals
    
#Get itr of any one experiment
itr = log['itr']
    
#Convert to np array
FID     = np.array(FID)
IS_mean = np.array(IS_mean)
IS_std  = np.array(IS_std)

#Start figure
plt.figure(figsize=[16,6])

#Last slice IS
plt.subplot(1,2,1)
ind = np.reshape(np.nanargmax(IS_mean, axis=1),[-1,1])
plt.errorbar(dim_z_vals, np.take_along_axis(IS_mean, ind, axis=1).squeeze(), 
             yerr=np.take_along_axis(IS_std, ind, axis=1).squeeze())
plt.xscale('log')
plt.title('Inception Score')
plt.ylabel('IS')
plt.xlabel('dim_z')

#Last slice FID
plt.subplot(1,2,2)
plt.semilogx(dim_z_vals, np.nanmin(FID, axis=1))
plt.title('Frechet Inception Distance')
plt.ylabel('FID')
plt.xlabel('dim_z')
plt.show()

In [ ]:
set_seed(0)
x = torch.randn(6, 2)

In [ ]:
# plt.errorbar(dim_z_vals, IS_mean[:,ind], yerr=IS_std[:,ind])


In [ ]:
x.shape

In [ ]:
dim_z

In [ ]:
x = -1
y = 1

In [ ]:
z = 2
if not (z == -1 or z > 0):
    print('fail')
else:
    print('pass')

In [ ]:
import torch
import numpy as np

In [ ]:
a = torch.randn(10)

In [ ]:
a[0] = float('nan')

In [ ]:
a